# Jupyter Lectures, First Year Project 2021

## Project 1 - Road collisions analysis, ITU Copenhagen

**Instructor: Michael Szell**

Course page: https://learnit.itu.dk/local/coursebase/view.php?ciid=590

This notebook contains all the code developed in the course lectures to wrangle and explore the data set from the project.

Contact: Michael Szell (misz@itu.dk)  
Created: 2021-01-29  
Last modified: 2021-01-29

<hr>

# Lecture 1: First data exploration

### Imports

In [1]:
import numpy as np
from pathlib import Path

### Constants

Constants are written all caps: https://www.python.org/dev/peps/pep-0008/#constants

In [4]:
PATH_NOTEBOOKS = Path.cwd()
PATH_REPOSITORY = PATH_NOTEBOOKS.parent
# absolute path tip: https://stackoverflow.com/questions/918154/relative-paths-in-python
# https://docs.python.org/3/library/pathlib.html

# the / is a way to join paths
PATH_DATA_RAW = PATH_REPOSITORY / "data/raw/"

F_ACCIDENTS_PATH = PATH_DATA_RAW / "Road Safety Data - Accidents 2019.csv"
F_CASUALTIES_PATH = PATH_DATA_RAW / "Road Safety Data - Casualties 2019.csv"
F_VEHICLES_PATH = PATH_DATA_RAW / "Road Safety Data- Vehicles 2019.csv"

# Open files for reading with correct encoding (handle first BOM char in files)
# https://stackoverflow.com/questions/17912307/u-ufeff-in-python-string
F_ACCIDENTS = open(F_ACCIDENTS_PATH, mode='r', encoding='utf-8-sig')
F_CASUALTIES = open(F_CASUALTIES_PATH, mode='r', encoding='utf-8-sig')
F_VEHICLES = open(F_VEHICLES_PATH,  mode='r', encoding='utf-8-sig')


In [5]:
print(PATH_REPOSITORY)

/mnt/b/Dokumenter/GitHub/fyp2021p01g04


### Load raw data

The data were downloaded from here on Jan 4th: https://data.gov.uk/dataset/road-accidents-safety-data  
That page was updated afterwards (Jan 8th), so local and online data may be inconsistent.

We first explore one data table, the accidents.

In [6]:
# Load data w/ specified types
# https://stackoverflow.com/questions/9534408/numpy-genfromtxt-produces-array-of-what-looks-like-tuples-not-a-2d-array-why
# names graps headers (first row)
data_accdnts = np.genfromtxt(F_ACCIDENTS, dtype=None, delimiter=",", names=True)
data_cslts = np.genfromtxt(F_CASUALTIES, dtype=None, delimiter=",", names=True)
data_vhcls = np.genfromtxt(F_VEHICLES, dtype=None, delimiter=",", names=True)

type(data_accdnts), type(data_cslts), type(data_vhcls)

(numpy.ndarray, numpy.ndarray, numpy.ndarray)

It is always good to start with a "sneak preview":

In [7]:
data_accdnts.shape, data_cslts.shape, data_vhcls.shape

((117536,), (153158,), (216381,))

Reminder and documentation on structured arrays:  
https://numpy.org/devdocs/user/basics.rec.html

#### Insight: Mixed variable types

In [8]:
data_accdnts[0]

(2019010128300, 528218, 180407, -0.153842, 51.508057, 1, 3, 2, 3, b'18/02/2019', 2, b'17:50', 1, b'E09000033', 3, 4202, 1, 30, 1, 2, 3, 4202, 0, 5, 1, 1, 1, 0, 0, 1, 3, b'E01004762')

Masks work!

In [9]:
mask = data_accdnts['Day_of_Week'] == 1
data_accdnts[mask].shape

(12935,)

Number of records

In [10]:
data_accdnts.shape[0], data_cslts.shape[0], data_vhcls.shape[0]

(117536, 153158, 216381)

Number of fields

In [11]:
# get first row and take len of tuple
len(data_accdnts[0]), len(data_cslts[0]), len(data_vhcls[0])

(32, 16, 23)

**"Data in the wild" puzzle: Why is the first field "\ufeffAccident_Index" and not "Accident_Index"?**

In [12]:
data_accdnts

array([(2019010128300, 528218, 180407, -0.153842, 51.508057,  1, 3, 2, 3, b'18/02/2019', 2, b'17:50',   1, b'E09000033', 3, 4202, 1, 30, 1,  2,  3, 4202,  0,  5, 1, 1, 1, 0, 0, 1, 3, b'E01004762'),
       (2019010152270, 530219, 172463, -0.127949, 51.436207,  1, 3, 2, 1, b'15/01/2019', 3, b'21:45',   9, b'E09000022', 3,   23, 2, 30, 0, -1, -1,    0, -1, -1, 4, 1, 1, 0, 0, 1, 3, b'E01003117'),
       (2019010155191, 530222, 182543, -0.124193, 51.526794,  1, 3, 2, 1, b'01/01/2019', 3, b'01:50',   2, b'E09000007', 4,  504, 6, 30, 3,  4,  6,    0,  0,  0, 4, 1, 1, 0, 0, 1, 1, b'E01000943'),
       ...,
       (2019984107219, 318544, 567087, -3.274645, 54.991684, 98, 3, 2, 1, b'21/06/2019', 6, b'15:30', 917, b'S12000006', 4,  723, 6, 60, 3,  4,  4,  721,  0,  0, 1, 1, 1, 0, 0, 2, 2, b''),
       (2019984107419, 336525, 584226, -2.997491, 55.148293, 98, 3, 1, 1, b'29/06/2019', 7, b'14:10', 917, b'S12000006', 6,  710, 6, 30, 3,  4,  6,  723,  0,  0, 1, 1, 1, 0, 0, 2, 2, b''),
       (        

Fields

Homework: Explore the other two tables the same way.

<hr>

In [11]:
data_cslts

array([(2019010128300, 1, 1, 1, 1, 58,  9, 3, 0, 0, 0, 0, 0, 9, 1, 2),
       (2019010128300, 1, 2, 2, 2, -1, -1, 3, 0, 0, 1, 0, 0, 9, 1, 5),
       (2019010128300, 1, 3, 2, 2, -1, -1, 3, 0, 0, 2, 0, 0, 9, 1, 5),
       ...,
       (2019984107219, 2, 1, 1, 1, 61,  9, 3, 0, 0, 0, 0, 0, 9, 3, 7),
       (2019984107419, 1, 1, 3, 1, 54,  8, 3, 5, 3, 0, 0, 0, 0, 3, 8),
       (           -1, 1, 1, 1, 1, 55,  8, 2, 0, 0, 0, 0, 0, 5, 1, 4)],
      dtype=[('Accident_Index', '<i8'), ('Vehicle_Reference', '<i4'), ('Casualty_Reference', '<i4'), ('Casualty_Class', '<i4'), ('Sex_of_Casualty', '<i4'), ('Age_of_Casualty', '<i4'), ('Age_Band_of_Casualty', '<i4'), ('Casualty_Severity', '<i4'), ('Pedestrian_Location', '<i4'), ('Pedestrian_Movement', '<i4'), ('Car_Passenger', '<i4'), ('Bus_or_Coach_Passenger', '<i4'), ('Pedestrian_Road_Maintenance_Worker', '<i4'), ('Casualty_Type', '<i4'), ('Casualty_Home_Area_Type', '<i4'), ('Casualty_IMD_Decile', '<i4')])

In [12]:
data_vhcls

array([(2019010128300, 1, 9, 0, -1, -1, -1, -1, -1, -1, -1,  4, -1, 6, 1, 58,  9,   -1, -1, -1, 2, 1, 2),
       (2019010128300, 2, 9, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, 6, 3, -1, -1,   -1, -1, -1, 2, 1, 2),
       (2019010152270, 1, 9, 0, 18, -1,  0, -1, -1, -1, -1,  1, -1, 6, 2, 24,  5,   -1, -1, -1, 3, 1, 3),
       ...,
       (2019984107219, 2, 9, 0, 18,  0,  1,  0,  0,  0,  0,  0,  1, 6, 1, 61,  9, 2967,  2,  5, 7, 3, 7),
       (2019984107419, 1, 9, 0,  7,  0,  6,  0,  0,  3,  0,  3,  1, 5, 1, 78, 11, 1597,  2,  6, 8, 3, 8),
       (           -1, 1, 5, 0, 16,  0,  0,  0,  0,  1,  0,  4,  1, 6, 1, 55,  8,  599,  1, 20, 4, 1, 4)],
      dtype=[('Accident_Index', '<i8'), ('Vehicle_Reference', '<i4'), ('Vehicle_Type', '<i4'), ('Towing_and_Articulation', '<i4'), ('Vehicle_Manoeuvre', '<i4'), ('Vehicle_LocationRestricted_Lane', '<i4'), ('Junction_Location', '<i4'), ('Skidding_and_Overturning', '<i4'), ('Hit_Object_in_Carriageway', '<i4'), ('Vehicle_Leaving_Carriageway', '<i4'), (

(134592,)

# Lecture 2: Command line wrangling and dealing with missing data

A faster way of getting basic insights into a new data set than by using numpy is by using command line tools.

Let's get a first overview using `head`. There are 3 data tables: Accidents, Casualties, and Vehicles.

### General insights

#### Link between data tables

Records between data tables are linked through their `Accident_Index`.

Looking at the first Accident_Index 2019010128300, we can see there seems to be a one-to-many relation between accident->casualty and accident->vehicle, meaning there can be multiple casualties and vehicles involved in one accident (makes sense).

https://en.wikipedia.org/wiki/One-to-many_(data_model)

#### Dimensions

Number of records

https://en.wikipedia.org/wiki/Wc_(Unix)

Number of fields (in first line)

https://www.geeksforgeeks.org/awk-command-unixlinux-examples/

See and count all fields

https://en.wikipedia.org/wiki/Tr_(Unix)
https://en.wikipedia.org/wiki/Nl_(Unix)

In [16]:
!head -1 "../data/raw/Road Safety Data - Accidents 2019.csv" | tr "," "\n" | nl

     1	﻿Accident_Index
     2	Location_Easting_OSGR
     3	Location_Northing_OSGR
     4	Longitude
     5	Latitude
     6	Police_Force
     7	Accident_Severity
     8	Number_of_Vehicles
     9	Number_of_Casualties
    10	Date
    11	Day_of_Week
    12	Time
    13	Local_Authority_(District)
    14	Local_Authority_(Highway)
    15	1st_Road_Class
    16	1st_Road_Number
    17	Road_Type
    18	Speed_limit
    19	Junction_Detail
    20	Junction_Control
    21	2nd_Road_Class
    22	2nd_Road_Number
    23	Pedestrian_Crossing-Human_Control
    24	Pedestrian_Crossing-Physical_Facilities
    25	Light_Conditions
    26	Weather_Conditions
    27	Road_Surface_Conditions
    28	Special_Conditions_at_Site
    29	Carriageway_Hazards
    30	Urban_or_Rural_Area
    31	Did_Police_Officer_Attend_Scene_of_Accident



### Sanity checks

Has each record the same number of fields?

https://shapeshed.com/unix-uniq/  
https://www.putorius.net/uniq-command-linux.html

How many duplicate lines are there? (If more than 0, there could be a problem)

More advanced stuff with `awk`: https://datafix.com.au/BASHing/2020-05-20.html

## Dealing with missing data

Using a masked array:  
https://numpy.org/devdocs/reference/maskedarray.baseclass.html#numpy.ma.MaskedArray

How many values in total?  
Which fields are missing?

Summary of missing values:

<hr>

# Lecture 3: Visual data exploration, Connecting tables, Association test

## Visual exploratory data analysis ("Plot your data")

### Bar plots of categorical variables

We were lucky because the categories 1,2,3 were "nice". But usually they aren't, so we need to explicitly map to integers:

Instead of copy-pasting code, let's write a function.

Typing the variable lookup manually is cumbersome. Can we read the excel directly?  
pandas can: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html

### Histograms of numerical variables

### Categorical scatterplots

Scatterplots are good for relating two numerical variables. If we have one numerical versus one categorical variable, we can do a box plot. But could we also visualize all data points? Yes: https://seaborn.pydata.org/tutorial/categorical.html

## Connecting tables with np.isin()

**Question: How many babies and toddlers died on UK roads in June 2019?**

Numpy has the fucntion isin() to select for a list of indices: https://numpy.org/doc/stable/reference/generated/numpy.isin.html

**Question: Who killed them?**

Homework

## Association test between two categorical variables 
**(Pearson $\chi^2$ test of independence)**

Inspired by:  
https://peterstatistics.com/CrashCourse/3-TwoVarUnpair/NomNom/NomNom-2a-Test.html  
https://bit.ly/3kbwKEL

**Let us ask: Is there a statistically significant association between accident severity and speed limit?**  
We ask because speed limit is something that the city government can regulate.

### Hypothesis testing

We are now in the realm of [Statistical hypothesis testing](https://en.wikipedia.org/wiki/Statistical_hypothesis_testing). In general, we must first state and compare two hypotheses:

- $H_0$ (null hypothesis): There is no statistically significant relationship between accident severity and speed limit.
- $H_\alpha$ (alternative hypothesis): There is a statistically significant relationship between accident severity and speed limit.

We must then 1) state+check statistical assumptions, 2) choose an appropriate test and test statistic $T$, 3) derive the distribution for the test statistic, 4) select a significance level $\alpha$, usually 0.01 or 0.05, 5) calculate the observed test statistic $t_{\mathrm obs}$, 6) calculate the [p-value](https://en.wikipedia.org/wiki/P-value). 

If the p-value $< \alpha$, then the null hypothesis will be rejected.

### Pearson $\chi^2$ test of independence

To test association between two categorical variables, one uses the [Pearson chi-square test of independence](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test). If the significance of this test (p-value) is below a significance level (typically 0.05), the two variables have a significant association.

The Pearson chi-square test should only be used if most cells have an expected count above 5, and the minimum expected count is at least 1.

We crosstabulate using pandas:
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html

The cross tabulation is also known as contingency table.

The idea is now to compare these observed values with expected values.  
The expected values can be calculated using:
\begin{equation*}
E_{i,j} = \frac{R_i \times C_j}{N}
\end{equation*}
The $E_{i,j}$ indicates the expected count in row i, column j. The $R_i$ is the row total of row i, and $C_j$ the column total of column j. The $N$ is the grand total.

That was the manual way of doing it. `chi2_contingency()` can do it for us automatically:

We now know that the association is significant, but how strong is it?  
Cramer's V, for example, can give an answer: https://en.wikipedia.org/wiki/Cram%C3%A9r%27s_V

The formula is:
\begin{equation*}
V=\sqrt{\frac{\chi^{2} / N}{\min (c-1, r-1)}}
\end{equation*}
where $c$ is the number of columns, $r$ is the number of rows.

Let us visualize this and make a human-readable plot and report below.

**Conclusion**

What about statistical tests for different combinations of numerical/categorical variables?
<img src="../references/flowchart-for-choosing-a-statistical-test.png" width="600px"/>

<hr>

# Lecture 4: Spatial filtering

## Filtering with external table

Let's select all rows for the city name:

We want to use this list of LSOA11 codes to restrict our accident data set.

Filter

Export

## Spatial filtering with shapely

Jupyter visualizes shapely objects!

Let's get all accident coordinates (from the whole UK)

`contains()` and `within()` check for point inclusion:

Limit vehicles and casualties to these AccidentIndices

What does it mean?  
See: https://www.computerhope.com/unix/udiff.htm

<hr>

# Lecture 5: Visualizing spatial data

Inspired by: https://alysivji.github.io/getting-started-with-folium.html

The heatmap is built with KDE:  
https://en.wikipedia.org/wiki/Kernel_density_estimation

We can also add automatic clusters: